In [ ]:
import glob
import yaml
import os


def round_nutil(nutil: float) -> float:
    if nutil < 0.525:
        return 0.5
    elif 0.525 <= nutil < 0.575:
        return 0.55
    elif 0.575 <= nutil < 0.625:
        return 0.6
    elif 0.625 <= nutil < 0.675:
        return 0.65
    elif 0.675 <= nutil < 0.725:
        return 0.7
    elif 0.725 <= nutil < 0.775:
        return 0.75
    elif 0.775 <= nutil < 0.825:
        return 0.8
    elif 0.825 <= nutil < 0.875:
        return 0.85
    elif 0.875 <= nutil < 0.925:
        return 0.9
    elif 0.925 <= nutil < 0.975:
        return 0.95
    elif 0.975 <= nutil:
        return 1.0


proposed_nutil_ar_map = {}
rm_nutil_ar_map = {}
greedy_nutil_ar_map = {}

nutils = [0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0]

for nutil in nutils:
    proposed_nutil_ar_map[nutil] = {'num_schedulable': 0, 'total': 0}
    rm_nutil_ar_map[nutil] = {'num_schedulable': 0, 'total': 0}
    greedy_nutil_ar_map[nutil] = {'num_schedulable': 0, 'total': 0}


def get_result_map(path: str) -> dict:
    if 'proposed' in path:
        return proposed_nutil_ar_map
    elif 'rm' in path:
        return rm_nutil_ar_map
    elif 'greedy' in path:
        return greedy_nutil_ar_map


for yaml_file in glob.glob(f'{os.environ["HOME"]}/2024_RTSS_WiP_Evaluation/outputs/*.yaml'):
    with open(yaml_file, 'r') as f:
        data = yaml.safe_load(f)

    if data['missed_job_id'] == 0:
        continue

    result_map = get_result_map(yaml_file)
    nutil = round_nutil(float(data['total_utilization']) / float(data['num_cores']))
    if data['missed_job_id'] == '2147483647':
        result_map[nutil]['num_schedulable'] += 1
    result_map[nutil]['total'] += 1


def calc_acceptance_ratio(result_map: dict) -> dict:
    for nutil in nutils:
        result_map[nutil] = float(result_map[nutil]['num_schedulable']
                                  ) / float(result_map[nutil]['total'])
    return result_map


proposed_nutil_ar_map = calc_acceptance_ratio(proposed_nutil_ar_map)
rm_nutil_ar_map = calc_acceptance_ratio(rm_nutil_ar_map)
greedy_nutil_ar_map = calc_acceptance_ratio(greedy_nutil_ar_map)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 4))
plt.plot(nutils, list(proposed_nutil_ar_map.values()), label='Extended GEDF',
         marker='s', markersize=6, markerfacecolor='none', color='crimson')
plt.plot(nutils, list(rm_nutil_ar_map.values()), label='RM', marker='o',
         markersize=6, markerfacecolor='none', color='darkblue')
plt.plot(nutils, list(greedy_nutil_ar_map.values()), label='Work-Conserving',
         marker='^', markersize=6, markerfacecolor='none', color='darkgreen')

plt.xlabel('Normalized Utilization', fontsize=16)
plt.ylabel('Acceptance Ratio', fontsize=16)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.legend(fontsize=14, loc='lower left')

plt.grid(True)
plt.savefig('acceptance_ratio.pdf')